In [3]:
import pandas as pd

In [4]:
df_orders = pd.read_csv('data/olist_orders_dataset.csv')
df_items = pd.read_csv('data/olist_order_items_dataset.csv')
df_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
df_customers = pd.read_csv('data/olist_customers_dataset.csv') 

In [5]:
colunas_data = [
    'order_purchase_timestamp', 
    'order_approved_at', 
    'order_delivered_carrier_date', 
    'order_delivered_customer_date', 
    'order_estimated_delivery_date'
]

for col in colunas_data:
    df_orders[col] = pd.to_datetime(df_orders[col])

In [8]:
df_orders = df_orders.rename(columns={
    'order_id': 'ID_Pedido',
    'customer_id': 'ID_Cliente',
    'order_status': 'Status_Pedido',
    'order_purchase_timestamp': 'Data_Compra',
    'order_approved_at': 'Data_Aprovacao',
    'order_delivered_carrier_date': 'Data_Envio_Transportadora',
    'order_delivered_customer_date': 'Data_Entrega',
    'order_estimated_delivery_date': 'Data_Previsao_Entrega'
})

In [ ]:
traducao_status = {
    'delivered': 'entregue',
    'shipped': 'enviado',
    'canceled': 'cancelado',
    'unavailable': 'indisponivel',
    'invoiced': 'faturado',
    'processing': 'em_processamento',
    'created': 'criado',
    'approved': 'aprovado'
}

df_orders['Status_Pedido'] = df_orders['Status_Pedido'].replace(traducao_status)


print("--- Distribuição de Status em Português ---")
print(df_orders['Status_Pedido'].value_counts(normalize=True))

In [ ]:
# Célula 6: Filtragem de Status

status_invalidos = ['cancelado', 'indisponivel']

df_orders_filtrado = df_orders[~df_orders['Status_Pedido'].isin(status_invalidos)].copy()


print(f"Total de Pedidos Original: {len(df_orders)}")
print(f"Total de Pedidos Válidos (Filtrados): {len(df_orders_filtrado)}")


print("\nVerificação dos Status Após Filtragem:")
print(df_orders_filtrado['Status_Pedido'].value_counts(normalize=True))

In [ ]:
# Célula 7: Preparação e Merge 1 (Clientes e Pedidos)

# 1. Renomear colunas do DataFrame de Clientes (df_customers)
df_customers = df_customers.rename(columns={
    'customer_id': 'ID_Cliente',
    'customer_unique_id': 'ID_Cliente_Unico',
    'customer_zip_code_prefix': 'CEP_Cliente',
    'customer_city': 'Cidade_Cliente',
    'customer_state': 'UF_Cliente'
})

# 2. Executar o Merge
# Tipo de merge (how): 'left', garantindo que todos os nossos 98.207 pedidos filtrados sejam mantidos.
df_base = pd.merge(
    df_orders_filtrado, 
    df_customers[['ID_Cliente', 'CEP_Cliente', 'Cidade_Cliente', 'UF_Cliente', 'ID_Cliente_Unico']], 
    on='ID_Cliente', 
    how='left'
)

# 3. Inspeção
print("--- Base de Pedidos após Merge com Clientes (df_base) ---")
df_base.info()

In [ ]:
# Célula 8: Criação da Base Logística (Apenas pedidos entregues)

# 1. Filtra a base para remover todos os pedidos onde a Data_Entrega é Nula (NaN)
df_logistica = df_base[df_base['Data_Entrega'].notna()].copy()

# 2. Reinspeção
print(f"Total de Pedidos Válidos (df_base): {len(df_base)}")
print(f"Total de Pedidos Entregues (df_logistica): {len(df_logistica)}")

# 3. Criação dos KPIs de Tempo (Feature Engineering - Início)

# Prazo de Entrega
df_logistica['Prazo_Entrega_Dias'] = (
    df_logistica['Data_Entrega'] - df_logistica['Data_Compra']
).dt.days

# Atraso em dias
df_logistica['Atraso_Dias'] = (
    df_logistica['Data_Entrega'] - df_logistica['Data_Previsao_Entrega']
).dt.days

# Criação da Flag de Atraso (is_late)
# Se a entrega foi DEPOIS da previsão, Atraso_Dias > 0, logo é Atrasado
df_logistica['Atrasado'] = (df_logistica['Atraso_Dias'] > 0).astype(int)

# 4. Exibe a Média Imediata (Insight Rápido)
print(f"\nMédia de Prazo de Entrega (dias): {df_logistica['Prazo_Entrega_Dias'].mean():.2f}")
print(f"Proporção de Pedidos Atrasados: {df_logistica['Atrasado'].mean():.2%}")

In [ ]:
# Célula 9: Agregação e Merge 2 (Itens)

# 1. Agrupar df_items por ID_Pedido e somar os valores para obter o total do pedido
df_itens_agregado = df_items.groupby('order_id').agg(
    Subtotal=('price', 'sum'),
    Frete_Total=('freight_value', 'sum'),
    Qtd_Itens=('order_item_id', 'count')
).reset_index()

# 2. Renomear a chave para o merge
df_itens_agregado = df_itens_agregado.rename(columns={'order_id': 'ID_Pedido'})

# 3. Merge 2: Juntar a base mestra com os dados agregados dos itens
# Chave: 'ID_Pedido'
df_base = pd.merge(df_base, df_itens_agregado, on='ID_Pedido', how='left')

# 4. Inspeção para verificar as novas colunas
print("--- Base Mestra após Merge com Itens (df_base) ---")
print(df_base[['ID_Pedido', 'Subtotal', 'Frete_Total', 'Qtd_Itens']].head())
print(df_base.info())

In [ ]:
# Célula 10: Agregação e Merge 3 (Pagamentos)

# 1. Renomear e selecionar colunas necessárias do df_payments
df_pagamentos_prep = df_payments.rename(columns={
    'order_id': 'ID_Pedido',
    'payment_type': 'Tipo_Pagamento',
    'payment_value': 'Valor_Pago'
})[['ID_Pedido', 'Tipo_Pagamento', 'Valor_Pago', 'payment_installments']]

# 2. Agrupar df_pagamentos_prep por ID_Pedido
df_pagamentos_agregado = df_pagamentos_prep.groupby('ID_Pedido').agg(
    Valor_Total_Pago=('Valor_Pago', 'sum'),
    Tipo_Pagamento_Principal=('Tipo_Pagamento', lambda x: x.mode()[0]), 
    Total_Parcelas=('payment_installments', 'sum')
).reset_index()

# 3. Merge 3: Juntar a base mestra com os dados agregados dos pagamentos
df_base = pd.merge(df_base, df_pagamentos_agregado, on='ID_Pedido', how='left')

# 4. Inspeção final da Base Mestra (apenas colunas financeiras)
print("--- Colunas Financeiras Finais no df_base ---")
print(df_base[['Subtotal', 'Frete_Total', 'Valor_Total_Pago', 'Tipo_Pagamento_Principal']].head())

In [ ]:
# Célula 11: Feature Engineering Financeira

# 1. Cálculo do Desconto (em Reais)
df_base['Desconto_Abs'] = (df_base['Subtotal'] + df_base['Frete_Total']) - df_base['Valor_Total_Pago']

# 2. Formalização da Receita Bruta (Valor que o cliente pagou)
df_base['Receita_Bruta'] = df_base['Valor_Total_Pago']

# 3. Cálculo do Ticket Médio por pedido 
# Ticket Médio = Receita Bruta / Quantidade de Pedidos 
# Como cada linha é um pedido único, é o Valor_Total_Pago.
df_base['Ticket_Medio'] = df_base['Valor_Total_Pago'] 

# 4. Cálculo do Desconto Percentual (Desconto_Abs / (Subtotal + Frete))
# Adicionamos uma pequena constante (0.001) ao divisor para evitar divisão por zero, caso Subtotal + Frete seja zero.
df_base['Desconto_Perc'] = (df_base['Desconto_Abs'] / (df_base['Subtotal'] + df_base['Frete_Total'] + 0.001)) * 100

# 5. Inspeção dos resultados
print("--- Novos KPIs Financeiros (Primeiras 5 linhas) ---")
print(df_base[['Receita_Bruta', 'Desconto_Abs', 'Desconto_Perc', 'Ticket_Medio']].head())

In [ ]:
# Célula 12: Arredondamento e Ajuste Fino do Desconto

# 1. Arredonda o desconto absoluto para 2 casas decimais.
df_base['Desconto_Abs'] = df_base['Desconto_Abs'].round(2)

# 2. Garante que qualquer valor negativo residual seja tratado como zero (sem 'desconto negativo').
df_base.loc[df_base['Desconto_Abs'] < 0, 'Desconto_Abs'] = 0

# 3. Recálculo do Desconto Percentual com o Desconto_Abs corrigido.
df_base['Desconto_Perc'] = (df_base['Desconto_Abs'] / (df_base['Subtotal'] + df_base['Frete_Total'] + 0.001)) * 100

# 4. Inspeção (filtrando pedidos com desconto > 0 para ver o efeito)
print("--- Verificação dos Descontos Corrigidos (Apenas pedidos com desconto) ---")
print(df_base[df_base['Desconto_Abs'] > 0][['Subtotal', 'Frete_Total', 'Valor_Total_Pago', 'Desconto_Abs', 'Desconto_Perc']].head())

In [ ]:
# Célula 13: Identificação de Outliers em Receita Bruta (Método IQR)

# 1. Definir a coluna a ser analisada
coluna_outlier = 'Receita_Bruta'

# 2. Calcular o Primeiro Quartil (Q1) e o Terceiro Quartil (Q3)
Q1 = df_base[coluna_outlier].quantile(0.25)
Q3 = df_base[coluna_outlier].quantile(0.75)
IQR = Q3 - Q1

# 3. Definir os limites inferior e superior para Outliers
# Limite Superior = Q3 + 1.5 * IQR
# Limite Inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
limite_inferior = Q1 - 1.5 * IQR

# 4. Contar e inspecionar os outliers (valores acima do limite superior)
outliers_count = df_base[df_base[coluna_outlier] > limite_superior].shape[0]

print(f"--- Análise de Outliers para {coluna_outlier} ---")
print(f"Q1 (25%): R$ {Q1:.2f}")
print(f"Q3 (75%): R$ {Q3:.2f}")
print(f"IQR: R$ {IQR:.2f}")
print(f"Limite Superior (Outlier): R$ {limite_superior:.2f}")
print(f"Total de Outliers acima do Limite: {outliers_count} pedidos")
print(f"Representa {outliers_count / len(df_base) * 100:.2f}% dos dados")

In [ ]:
# Célula 14: Implementação do Capping (Limite de Outlier)

# O limite superior calculado foi R$ 348.88
LIMITE_SUPERIOR_RECEITA = 348.88

# Aplicar o capping: onde a Receita_Bruta for maior que o limite,
# substituímos o valor da Receita_Bruta (e Ticket_Medio) pelo limite.
df_base.loc[df_base['Receita_Bruta'] > LIMITE_SUPERIOR_RECEITA, 'Receita_Bruta'] = LIMITE_SUPERIOR_RECEITA
df_base.loc[df_base['Receita_Bruta'] > LIMITE_SUPERIOR_RECEITA, 'Ticket_Medio'] = LIMITE_SUPERIOR_RECEITA


# Verificação: Exibe a nova média (deve ser menor que a média original)
nova_media = df_base['Receita_Bruta'].mean()
print(f"Novo Ticket Médio (Receita Bruta após capping): R$ {nova_media:.2f}")

In [ ]:
# Célula 15: Identificação de Outliers em Frete_Total (Método IQR)

# 1. Definir a coluna a ser analisada
coluna_outlier_frete = 'Frete_Total'

# 2. Calcular o Primeiro Quartil (Q1) e o Terceiro Quartil (Q3)
Q1_frete = df_base[coluna_outlier_frete].quantile(0.25)
Q3_frete = df_base[coluna_outlier_frete].quantile(0.75)
IQR_frete = Q3_frete - Q1_frete

# 3. Definir os limites inferior e superior
limite_superior_frete = Q3_frete + 1.5 * IQR_frete
limite_inferior_frete = Q1_frete - 1.5 * IQR_frete

# 4. Contar e inspecionar os outliers (valores acima do limite superior)
outliers_frete_count = df_base[df_base[coluna_outlier_frete] > limite_superior_frete].shape[0]

print(f"--- Análise de Outliers para {coluna_outlier_frete} ---")
print(f"Q1 (25%): R$ {Q1_frete:.2f}")
print(f"Q3 (75%): R$ {Q3_frete:.2f}")
print(f"Limite Superior (Outlier): R$ {limite_superior_frete:.2f}")
print(f"Total de Outliers acima do Limite: {outliers_frete_count} pedidos")
print(f"Representa {outliers_frete_count / len(df_base) * 100:.2f}% dos dados")

In [ ]:
# Célula 16: Implementação do Capping no Frete_Total

# O limite superior calculado foi R$ 39.34
LIMITE_SUPERIOR_FRETE = 39.34

# Aplicar o capping: onde o Frete_Total for maior que o limite,
# substituímos o valor pelo limite (R$ 39.34).
df_base.loc[df_base['Frete_Total'] > LIMITE_SUPERIOR_FRETE, 'Frete_Total'] = LIMITE_SUPERIOR_FRETE

# Verificação: Exibe o novo Custo Médio de Frete
nova_media_frete = df_base['Frete_Total'].mean()
print(f"Novo Custo Médio de Frete (após capping): R$ {nova_media_frete:.2f}")